<a href="https://colab.research.google.com/gist/SK124/ba3c3cca86f5d6cb788d72b57892f9db/drugagentrl-mat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" Imports neccesary for installing RDkit """
 
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!#cp '/content/drive/My Drive/Miniconda3-latest-Linux-x86_64.sh' /content/
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
sys.path.append('/usr/local/lib/python3.7/site-packages/rdkit')
%cd /usr/local/lib/python3.7/site-packages/

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - ca-certificates==2020.1.1=0
    - certifi==2020.4.5.1=py37_0
    - cffi==1.14.0=py37he30daa8_1
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.1=py37h7b6447c_0
    - conda==4.8.3=py37_0
    - cryptography==2.9.2=py37h1ba5d50_0
    - idna==2.9=py_1
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.3=he6710b0_1
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1g=h7b6447c_0
    - pip==20.0.2=py37_3
    - pycosat==0.6.3=py37h7b6447c_0
    - pycparser==2.20=py_0
    - pyopenssl==19.1.0=py37_0
    - pysocks==1.7.1=py37_0
    - python==3.7.7=hcff3b4d_5
    - readline==8.0=h7b6447c_0
    - requests==2.23.0=py37_0
    - ruamel_yaml==0.15.87=p

In [ ]:
""" Imports for preprocessing,loss calculaitons and other utilities """

import re
import os
import pandas as pd
import numpy as np
import random
import pickle
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CosineSimilarity
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MolFromSmiles
from rdkit.Chem import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import RDConfig
from tqdm import tqdm

In [ ]:
%cd /content

/content


In [ ]:
""" Loading corpus into the memory ; It is a CSV File """
path='/content/drive/My Drive/De NovoDrug/full_data_final.csv'
df=pd.read_csv(path)
df.head()

,col
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...


In [ ]:
def csv2txt(dataframe):
 """ CSV needs to be converted to a text file with each SMILES string in a new line """
 with open('drug.txt','w') as f:
       for i in range(len(dataframe)):
           text=dataframe['col'][i]
           f.write(text+'\n')

In [ ]:
csv2txt(df)

In [ ]:
def txt2list(fname):
# " Takes the molecules in text file to and store it in list "
  smiles = []
  with open(fname, 'r') as f:
     for line in f:
         smiles.append(line.split()[0])
  return smiles

In [ ]:
smiles=txt2list('drug.txt')

In [ ]:
def replace_halogen(string):
    """Regex to replace Br and Cl with single letters"""
    br = re.compile('Br')
    cl = re.compile('Cl')
    string = br.sub('R', string)
    string = cl.sub('L', string)

    return string


In [ ]:
def construct_vocabulary(smiles_list,fname):
    #"""Returns all the characters present in a SMILES file.
    #   Uses regex to find characters/tokens of the format '[x]'."""
    add_chars = set()
    for i, smiles in enumerate(smiles_list):
        regex = '(\[[^\[\]]{1,10}\])'

       # """
        #Regex Explaination : \ signifies a special sequence inside a specific set [] of characters for eg.([...CC[NH+]COCN...]) so \[...] , which can contain [NH+] like molecules inside it which is taken care by  the ... as explained in the following  
        #To capture such [NH+],[C@@H],[...] molecules, ^ (start with operator) is used, which is to say such a token is going to start, inside which we give the \[\] to denote a specific sequence which will start with a '[' and end with ']' ie.[NH+] or [C@@H], here \ is used to say escape the inbetween enitities(C@@H,NH+) ie dont split them
        #There can be numbers 1,2....10 to denote begining of rings within the sequence like CCCcc1n.... to capture these numbers {1,10} ie all numbers between 1 to 10, is used
        #INPUT SMILE ----> 'CCC[C@@H][NH3+][O+][O-][OH+][P+]COCCCcc1nnn' 
        #char_list = ['CCC','[C@@H]','[NH3+]','[O+]','[O-]','[OH+]','[P+]','COCCCcc1nnn']
         
        #"""

        smiles = replace_halogen(smiles)
        char_list = re.split(regex, smiles)
        for char in char_list:
        #"There are characters like [NH+] which need to considered as single token rather than '[', 'N','H','+',']'"
            if char.startswith('['):
                add_chars.add(char)

            else:
            #"chars inside charlist need to be split as well i.e. COCCCcc1nnn need to split into tokens like 'C','O','1','c','n' "
                chars = [unit for unit in char]
                [add_chars.add(unit) for unit in chars]

    print("Number of characters: {}".format(len(add_chars)))
    with open(fname, 'w') as f:
        for char in add_chars:
            f.write(char + "\n")
    return add_chars


In [ ]:
construct_vocabulary(smiles,'vocab.txt')

Number of characters: 58


{'#',
 '(',
 ')',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '=',
 'C',
 'F',
 'I',
 'L',
 'N',
 'O',
 'P',
 'R',
 'S',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[CH-]',
 '[CH2-]',
 '[N+]',
 '[N-]',
 '[NH+]',
 '[NH-]',
 '[NH2+]',
 '[NH3+]',
 '[O+]',
 '[O-]',
 '[OH+]',
 '[P+]',
 '[P@@H]',
 '[P@@]',
 '[P@]',
 '[PH2]',
 '[S+]',
 '[S-]',
 '[S@@]',
 '[S@]',
 '[SH+]',
 '[n+]',
 '[n-]',
 '[nH+]',
 '[nH]',
 '[o+]',
 '[s+]',
 '\\',
 'c',
 'n',
 'o',
 's'}

In [ ]:
class Vocabulary(object):
   # """A class for handling encoding/decoding from SMILES to an array of indices"""
    def __init__(self, init_from_file=None, max_length=140):
    #    "E and G are used to signify ending and begining of seqeunce"
        self.special_tokens = ['E', 'G']
      #   "Other tokens from SMILES grammer needs to be added"
        self.additional_chars = set()
        self.chars = self.special_tokens
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.reversed_vocab = {v: k for k, v in self.vocab.items()}
        self.max_length = max_length
        if init_from_file: 
       #     " vocab.txt file which contains all the tokens is given as a input which initiates the vocabulary for the language model" 
          self.init_from_file(init_from_file)

    def encode(self, char_list):
      #  """Takes a list of characters (eg '[NH]') and encodes to array of indices"""
        smiles_matrix = np.zeros(len(char_list), dtype=np.float32)
        for i, char in enumerate(char_list):
       #    "2D Matrix : All the tokens are converted from SMILES to integers"
            smiles_matrix[i] = self.vocab[char]
        return smiles_matrix

    def decode(self, matrix):
      #  """Takes an array of indices and returns the corresponding SMILES"""
        chars = []
        for i in matrix:
            if i == self.vocab['E']: break
          #   "If E token which is the End token is reached, the process terminates"
            chars.append(self.reversed_vocab[i])
        smiles = "".join(chars)
        smiles = smiles.replace("L", "Cl").replace("R", "Br")
        return smiles

    def tokenize(self, smiles):
       # """Takes a SMILES and return a list of characters/tokens"""
        regex = '(\[[^\[\]]{1,6}\])'
        smiles = replace_halogen(smiles)
        char_list = re.split(regex, smiles)
        tokenized = []
       # "SMILES Sequence needs to be converted to tokens for preprocessing, this is the same code we had in Vocabulary class with few modifns."
        for char in char_list:
            if char.startswith('['):
                tokenized.append(char)
            else:
                chars = [unit for unit in char]
                [tokenized.append(unit) for unit in chars]
        # "E token is added to signify the ending of a token"
        tokenized.append('E')
        return tokenized

    def add_characters(self, chars):
     #   """Adds characters to the vocabulary"""
        for char in chars:
            self.additional_chars.add(char)
        char_list = list(self.additional_chars)
        char_list.sort()
        self.chars = char_list + self.special_tokens
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.reversed_vocab = {v: k for k, v in self.vocab.items()}

    def init_from_file(self, file):
       # """Takes a file containing \n separated characters to initialize the vocabulary"""
        with open(file, 'r') as f:
      #     "All characters from vocab.txt file which has all the unique tokens is taken and added to the Vocabukary of the model"
            chars = f.read().split()
        self.add_characters(chars)

    def __len__(self):
        return len(self.chars)

    def __str__(self):
        return "Vocabulary containing {} tokens: {}".format(len(self), self.chars)


In [ ]:
class MolData(Dataset):
    """Custom PyTorch Dataset that takes a file containing SMILES.
        Args:
                fname : path to a file containing \n separated SMILES.
                voc   : a Vocabulary instance
        Returns:
                A custom PyTorch dataset for training the Prior.
    """
    def __init__(self, fname, voc):
        self.voc = voc
        self.smiles = []
        with open(fname, 'r') as f:
            for line in f:
                self.smiles.append(line.split()[0])

    def __getitem__(self, i): 
        mol = self.smiles[i]
        tokenized = self.voc.tokenize(mol)
        encoded = self.voc.encode(tokenized)
        return Variable(encoded)

    def __len__(self):
        return len(self.smiles)

    def __str__(self):
        return "Dataset containing {} structures.".format(len(self))

    @classmethod
    def collate_fn(cls, arr):
        """Function to take a list of encoded sequences and turn them into a batch"""
        max_length = max([seq.size(0) for seq in arr])
        """ For every batch lets say 64, 64 sequences will be taken an the maximum length of the sequence from that batch woul be max_length 
        and the shorter seqeunces will be padded till the max_length before passing it to the GRU """
        collated_arr = Variable(torch.zeros(len(arr), max_length))
        for i, seq in enumerate(arr):
            collated_arr[i, :seq.size(0)] = seq
        return collated_arr


In [ ]:
def Variable(tensor):
    """Wrapper for torch.autograd.Variable that also accepts
       numpy arrays directly and automatically assigns it to
       the GPU if availale else CPU."""
    if isinstance(tensor, np.ndarray):
        tensor = torch.from_numpy(tensor)
    if torch.cuda.is_available():
        return torch.autograd.Variable(tensor).cuda()
    return torch.autograd.Variable(tensor)

def decrease_learning_rate(optimizer, decrease_by=0.01):
    """Multiplies the learning rate of the optimizer by 1 - decrease_by"""
    for param_group in optimizer.param_groups:
        param_group['lr'] *= (1 - decrease_by)

def seq_to_smiles(seqs, voc):
    """Takes an output sequence from the RNN and returns the
       corresponding SMILES."""
    smiles = []
    for seq in seqs.cpu().numpy():
        smiles.append(voc.decode(seq))
    return smiles

def fraction_valid_smiles(smiles):
    """Takes a list of SMILES and returns fraction valid."""
    i = 0
    for smile in smiles:
        if Chem.MolFromSmiles(smile):
            i += 1
    return i / len(smiles)

def unique(arr):
    """Finds unique rows in arr and return their indices"""
    arr = arr.cpu().numpy()
    arr_ = np.ascontiguousarray(arr).view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[1])))
    _, idxs = np.unique(arr_, return_index=True)
    if torch.cuda.is_available():
        return torch.LongTensor(np.sort(idxs)).cuda()
    return torch.LongTensor(np.sort(idxs))

In [ ]:
class Experience(object):
    """Class for prioritized experience replay that remembers the highest scored sequences
       seen and samples from them with probabilities relative to their scores.
       To perform experience replay we store the agent's experiences """
    def __init__(self, voc, max_size=100):
        self.memory = []
        " Memory stores SMILE decoded seq,respective score,NLL_likelihood "
        self.max_size = max_size
        self.voc = voc

    def add_experience(self, experience):
        """Experience should be a list of (smiles, score, prior likelihood) tuples"""
        self.memory.extend(experience)
        if len(self.memory)>self.max_size:
            # Remove duplicates as sampling might prouce same moelcule occasionally
            idxs, smiles = [], []
            for i, exp in enumerate(self.memory):
                if exp[0] not in smiles:
                    idxs.append(i)
                    smiles.append(exp[0])
            self.memory = [self.memory[idx] for idx in idxs]
            # Retain only the highest scores
            self.memory.sort(key = lambda x: x[1], reverse=True)
            self.memory = self.memory[:self.max_size]
            print("\nBest score in memory: {:.2f}".format(self.memory[0][1]))

    def sample(self, n):
        """Sample a batch size n of experience"""
        if len(self.memory)<n:
            raise IndexError('Size of memory ({}) is less than requested sample ({})'.format(len(self), n))
        else:
            scores = [x[1] for x in self.memory]
            sample = np.random.choice(len(self), size=n, replace=False, p=scores/np.sum(scores))
            sample = [self.memory[i] for i in sample]
            smiles = [x[0] for x in sample]
            scores = [x[1] for x in sample]
            prior_likelihood = [x[2] for x in sample]
        tokenized = [self.voc.tokenize(smile) for smile in smiles]
        encoded = [Variable(self.voc.encode(tokenized_i)) for tokenized_i in tokenized]
        encoded = MolData.collate_fn(encoded)
        return encoded, np.array(scores), np.array(prior_likelihood)

    def initiate_from_file(self, fname, scoring_function, Prior):
        """Adds experience from a file with SMILES
           Needs a scoring function and an RNN to score the sequences.
           Using this feature means that the learning can be very biased
           and is typically advised against."""
        with open(fname, 'r') as f:
            smiles = []
            for line in f:
                smile = line.split()[0]
                if Chem.MolFromSmiles(smile):
                    smiles.append(smile)
        scores = scoring_function(smiles)
        tokenized = [self.voc.tokenize(smile) for smile in smiles]
        encoded = [Variable(self.voc.encode(tokenized_i)) for tokenized_i in tokenized]
        encoded = MolData.collate_fn(encoded)
        prior_likelihood, _ = Prior.likelihood(encoded.long())
        prior_likelihood = prior_likelihood.data.cpu().numpy()
        new_experience = zip(smiles, scores, prior_likelihood)
        self.add_experience(new_experience)

    def print_memory(self, path):
        """Prints the memory."""
        print("\n" + "*" * 80 + "\n")
        print("         Best recorded SMILES: \n")
        print("Score     Prior log P     SMILES\n")
        with open(path, 'w') as f:
            f.write("SMILES Score PriorLogP\n")
            for i, exp in enumerate(self.memory[:100]):
                if i < 50:
                    print("{:4.2f}   {:6.2f}        {}".format(exp[1], exp[2], exp[0]))
                    f.write("{} {:4.2f} {:6.2f}\n".format(*exp))
        print("\n" + "*" * 80 + "\n")

    def __len__(self):
        return len(self.memory)


In [ ]:
class MultiGRU(nn.Module):
    """ Implements a three layer GRU cell including an embedding layer
       and an output linear layer back to the size of the vocabulary"""
    def __init__(self, voc_size):
        super(MultiGRU, self).__init__()
        self.embedding = nn.Embedding(voc_size, 128)
        self.gru_1 = nn.GRUCell(128, 512)
        self.gru_2 = nn.GRUCell(512, 512)
        self.gru_3 = nn.GRUCell(512, 512)
        self.linear = nn.Linear(512, voc_size)

    def forward(self, x, h):
        x = self.embedding(x)
        h_out = Variable(torch.zeros(h.size()))
        x = h_out[0] = self.gru_1(x, h[0])
        x = h_out[1] = self.gru_2(x, h[1])
        x = h_out[2] = self.gru_3(x, h[2])
        x = self.linear(x)
        return x, h_out

    def init_h(self, batch_size):
        # Initial cell state is zero
        return Variable(torch.zeros(3, batch_size, 512))

class RNN():
    """Implements the Prior and Agent RNN. Needs a Vocabulary instance in
    order to determine size of the vocabulary and index of the End (E) token"""
    def __init__(self, voc):
        self.rnn = MultiGRU(voc.vocab_size)
        if torch.cuda.is_available():
            self.rnn.cuda()
        self.voc = voc

    def likelihood(self, target):
        """
            Retrieves the likelihood of a given sequence
            Args:
                target: (batch_size * sequence_length) A batch of sequences
            Outputs:
                log_probs : (batch_size) Log likelihood for each example*
                entropy: (batch_size) The entropies for the sequences. Not
                                      currently used.
        """
        batch_size, seq_length = target.size()
        start_token = Variable(torch.zeros(batch_size, 1).long())
        start_token[:] = self.voc.vocab['G']
        x = torch.cat((start_token, target[:, :-1]), 1)
        h = self.rnn.init_h(batch_size)

        log_probs = Variable(torch.zeros(batch_size))
        entropy = Variable(torch.zeros(batch_size))
        for step in range(seq_length):
        #   """ For every step from 0 to sequnce length, we pass the GRU seqeunce based on the timestep and hidden state and get logits and 
         # hidden state after taking that step
         # logits is a vector of 'D' dimension where D = voc_size with scores on what should be the next token, which is converted to 
        #  log_probs using log_softmax and sent to the NLL Loss function which compares the next predicted word by RNN and Ground Truth and calculates 
        #  the loss per sequence per timestep which is accumulated under log_probs and returns total loss after end time step """
            logits, h = self.rnn(x[:, step], h)
            log_prob = F.log_softmax(logits)
            prob = F.softmax(logits)
            log_probs += NLLLoss(log_prob, target[:, step])
            entropy += -torch.sum((log_prob * prob), 1)
        return log_probs, entropy

    def sample(self, batch_size, max_length=140):
        """
            Sample a batch of sequences
            Args:
                batch_size : Number of sequences to sample 
                max_length:  Maximum length of the sequences
            Outputs:
            seqs: (batch_size, seq_length) The sampled sequences.
            log_probs : (batch_size) Log likelihood for each sequence.
            entropy: (batch_size) The entropies for the sequences. Not
                                    currently used.
        """
        start_token = Variable(torch.zeros(batch_size).long())
        start_token[:] = self.voc.vocab['G']
        h = self.rnn.init_h(batch_size)
        x = start_token

        sequences = []
        log_probs = Variable(torch.zeros(batch_size))
        finished = torch.zeros(batch_size).byte()
        entropy = Variable(torch.zeros(batch_size))
        if torch.cuda.is_available():
            finished = finished.cuda()

        for step in range(max_length):
        #   """ After language model learns the semantic syntax, we need to check its perfomance which is why we sample from it.
        #  The code here is exactly similar to the code in likeilhood function """ 
            logits, h = self.rnn(x, h)
            prob = F.softmax(logits)
            log_prob = F.log_softmax(logits)
        #    """ torch.multinomial returns a tensor where each row contains 1 index sampled from the multinomial probability distribution 
         ##   located in the corresponding row of tensor input.
        #    """
            x = torch.multinomial(prob,1).view(-1)
            sequences.append(x.view(-1, 1))
            log_probs +=  NLLLoss(log_prob, x)
            entropy += -torch.sum((log_prob * prob), 1)

            x = Variable(x.data)
       #      """ Sampling terminates when E token is spit out of GRU """
            EOS_sampled = (x == self.voc.vocab['E']).data
            finished = torch.ge(finished + EOS_sampled, 1)
            if torch.prod(finished) == 1: break

        sequences = torch.cat(sequences, 1)
        return sequences.data, log_probs, entropy

def NLLLoss(inputs, targets):
    """
        Custom Negative Log Likelihood loss that returns loss per example,
        rather than for the entire batch.
        Args:
            inputs : (batch_size, num_classes) *Log probabilities of each class*
            targets: (batch_size) *Target class index*
        Outputs:
            loss : (batch_size) *Loss for each example*
    """

    if torch.cuda.is_available():
        target_expanded = torch.zeros(inputs.size()).cuda()
    else:
        target_expanded = torch.zeros(inputs.size())

    target_expanded.scatter_(1, targets.contiguous().view(-1, 1).data, 1.0)
    loss = Variable(target_expanded) * inputs
    loss = torch.sum(loss, 1)
    return loss

In [ ]:
def get_scoring_function(scoring_function, num_processes=None, **kwargs):
    """Function that initializes and returns a scoring function by name"""
    scoring_function_classes = [MAT,tanimoto]
    scoring_functions = [f.__name__ for f in scoring_function_classes]
    scoring_function_class = [f for f in scoring_function_classes if f.__name__ == scoring_function][0]

    if scoring_function not in scoring_functions:
        raise ValueError("Scoring function must be one of {}".format([f for f in scoring_functions]))

    for k, v in kwargs.items():
        if k in scoring_function_class.kwargs:
            setattr(scoring_function_class, k, v)

    if num_processes == 0:
        return Singleprocessing(scoring_function=scoring_function_class)
    return Multiprocessing(scoring_function=scoring_function, num_processes=num_processes)

In [ ]:
%cd /content
!git clone https://github.com/ardigen/MAT.git
import pandas as pd
import torch
%cd /content/MAT
os.chdir('src')
from featurization.data_utils import load_data_from_smiles, construct_loader
batch_size = 1

# Formal charges are one-hot encoded to keep compatibility with the pre-trained weights.
# If you do not plan to use the pre-trained weights, we recommend to set one_hot_formal_charge to False.
client='C1=CC=CC=C1'
batch_size = 1
client_list=[client]
label=list([float(1)])

!cp /content/drive/'/My Drive'/transform.py  /content/MAT/src
from transform import make_models

#d_atom = X[0][0].shape[1]  # It depends on the used featurization.

model_params = {
    'd_atom': 28,
    'd_model': 1024,
    'N': 8,
    'h': 16,
    'N_dense': 1,
    'lambda_attention': 0.33, 
    'lambda_distance': 0.33,
    'leaky_relu_slope': 0.1, 
    'dense_output_nonlinearity': 'relu', 
    'distance_matrix_kernel': 'exp', 
    'dropout': 0.0,
    
}

model = make_models(**model_params)
model
!cp '/content/drive/My Drive/De NovoDrug/pretrained_weights.pt' /content/MAT/src
pretrained_name = 'pretrained_weights.pt'  # This file should be downloaded first (See README.md).
pretrained_state_dict = torch.load(pretrained_name)
model_state_dict = model.state_dict()
for name, param in pretrained_state_dict.items():
    if 'generator' in name:
         continue
    if isinstance(param, torch.nn.Parameter):
        param = param.data
    model_state_dict[name].copy_(param)

In [ ]:
def MATSCORE(smiles):
      specimen=smiles
      batch_size = 1
      client_list=[specimen]
      label=list([float(1)])
      X, Y = load_data_from_smiles(client_list,label, one_hot_formal_charge=True)
      data_loader = construct_loader(X, Y, batch_size)

      for batch in data_loader:
          adjacency_matrix, node_features, distance_matrix, y = batch
          batch_mask = torch.sum(torch.abs(node_features), dim=-1) != 0
          output = model(node_features, batch_mask, adjacency_matrix, distance_matrix,None)   
          
      x=output
      mask=batch_mask
      mask = mask.unsqueeze(-1).float()
      out_masked = x * mask
      out_sum = x.sum(dim=1)
      mask_sum = mask.sum(dim=(1))
      out_avg_pooling = out_sum / mask_sum
      return out_avg_pooling

In [ ]:
class MAT():

  kwargs=["k","client_struct"]
  k=0.95
  cos = nn.CosineSimilarity(dim=1, eps=1e-6)
  client_struct='C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl'
  def __init__(self):
    self.client=MATSCORE(self.client_struct)
  def __call__(self, smile):
        mol = Chem.MolFromSmiles(smile)
        if mol:
            fp = MATSCORE(smile)
            score = self.cos(self.client, fp)
            score = min(score, self.k) / self.k
            return float(score)
        return 0.0




In [ ]:
class tanimoto():
    """ Scores structures based on Tanimoto similarity to a query structure.
        Scores are only scaled up to k=(0,1), after which no more reward is given."""

    kwargs = ["k", "query_structure"]
    k = 0.7
    #query_structure = "Cc1ccc(cc1)c2cc(nn2c3ccc(cc3)S(=O)(=O)N)C(F)(F)F"
    #query_structure ="C1=CC=C(C=C1)Cl"
    query_structure="C1=CC=C(C=C1)Cl"

    def __init__(self):
        query_mol = Chem.MolFromSmiles(self.query_structure)
        self.query_fp = AllChem.GetMorganFingerprint(query_mol, 2, useCounts=True, useFeatures=True)

    def __call__(self, smile):
        mol = Chem.MolFromSmiles(smile)
        if mol:
            fp = AllChem.GetMorganFingerprint(mol, 2, useCounts=True, useFeatures=True)
            score = DataStructs.TanimotoSimilarity(self.query_fp, fp)
            score = min(score, self.k) / self.k
            return float(score)
        return 0.0

In [ ]:
class Singleprocessing():
    """Adds an option to not spawn new processes for the scoring functions, but rather
       run them in the main process."""
    def __init__(self, scoring_function=None):
        self.scoring_function = scoring_function()
    def __call__(self, smiles):
        scores = [self.scoring_function(smile) for smile in smiles]
        return np.array(scores, dtype=np.float32)


In [ ]:
import os
from shutil import copyfile
import sys
import pexpect
import threading
from rdkit import rdBase
from rdkit.Chem import AllChem
rdBase.DisableLog('rdApp.error')


In [ ]:
if __name__ == "__main__":

    smiles_file = '/content/drug.txt'
    print("Reading smiles...")
    smiles_list = smiles
    print("Constructing vocabulary...")
    voc_chars = construct_vocabulary(smiles_list, "vocab.txt")

Reading smiles...
Constructing vocabulary...
Number of characters: 58


In [ ]:
def train_agent(restore_prior_from='/content/drive/My Drive/De NovoDrug/50_epoch.ckpt',
                restore_agent_from='/content/drive/My Drive/De NovoDrug/50_epoch.ckpt',
                scoring_function='MAT',
                scoring_function_kwargs=None,
                save_dir=None, learning_rate=0.0005,
                batch_size=64, n_steps=3000,
                num_processes=0, sigma=60,
                experience_replay=0):
  

    #""" Loads the vocabulary, initalizes Prior and Agent Language Models, Calculates scores,loss, experience score ,loss (RL) and writes the 
   # valid and similarly scored molecules wrt the molecule given into a text file during training """
    

    voc = Vocabulary(init_from_file="/content/vocab.txt")

    start_time = time.time()

    Prior = RNN(voc)
    Agent = RNN(voc)


    """ By default restore Agent to same model as Prior, but can restore from already trained Agent too.
     Saved models are partially on the GPU, but if we dont have cuda enabled we can remap these
     to the CPU."""
    if torch.cuda.is_available():
        Prior.rnn.load_state_dict(torch.load('/content/drive/My Drive/De NovoDrug/50_epoch.ckpt'))
        Agent.rnn.load_state_dict(torch.load(restore_prior_from))
    else:
        Prior.rnn.load_state_dict(torch.load('/content/drive/My Drive/De NovoDrug/50_epoch.ckpt', map_location=lambda storage, loc: storage))
        Agent.rnn.load_state_dict(torch.load(restore_agent_from, map_location=lambda storage, loc: storage))

    # We dont need gradients with respect to Prior
    for param in Prior.rnn.parameters():
        param.requires_grad = False

    optimizer = torch.optim.Adam(Agent.rnn.parameters(), lr=0.0005)

    # Scoring_function
    scoring_function = get_scoring_function(scoring_function=scoring_function, num_processes=num_processes,k=0.99,query_structure=MAT)
  


   # """" For policy based RL, we normally train on-policy and correct for the fact that more likely actions
   # occur more often (which means the agent can get biased towards them).
   # Using experience replay is therefor not as theoretically sound as it is for value based RL, but it seems to work well."""
    experience = Experience(voc)

    print("Model initialized, starting training...")

    for step in range(n_steps):

        # Sample from Agent
        seqs, agent_likelihood, entropy = Agent.sample(batch_size)

        # Remove duplicates, ie only consider unique seqs
        unique_idxs = unique(seqs)
        seqs = seqs[unique_idxs]
        agent_likelihood = agent_likelihood[unique_idxs]
        entropy = entropy[unique_idxs]

        # Get prior likelihood and score
        prior_likelihood, _ = Prior.likelihood(Variable(seqs))
        smiles = seq_to_smiles(seqs, voc)
        score = scoring_function(smiles)

        # Calculate augmented likelihood
        augmented_likelihood = prior_likelihood + sigma * Variable(score)
        loss = torch.pow((augmented_likelihood - agent_likelihood), 2)

        # Experience Replay
        # First sample
        if experience_replay and len(experience)>4:
        #""" After adding experience from first n_step, we sample from them and take the 4 samples (using np.random.choice) and their scores,losses and add it with agent's loss,likelihood."""
        #        """Reason : Experience replay enables reinforcement learning agents to memorize and reuse past experiences, just as humans replay memories for the situation at hand."""
         #       """Contemporary off-policy algorithms either replay past experiences uniformly or utilize a rule- based replay strategy, which may be sub-optimal."""
            exp_seqs, exp_score, exp_prior_likelihood = experience.sample(4)
            exp_agent_likelihood, exp_entropy = Agent.likelihood(exp_seqs.long())
            exp_augmented_likelihood = exp_prior_likelihood + sigma * exp_score
            exp_loss = torch.pow((Variable(exp_augmented_likelihood) - exp_agent_likelihood), 2)
            loss = torch.cat((loss, exp_loss), 0)
            agent_likelihood = torch.cat((agent_likelihood, exp_agent_likelihood), 0)

        # Then add new experience
        prior_likelihood = prior_likelihood.data.cpu().numpy()
        new_experience = zip(smiles, score, prior_likelihood)
        experience.add_experience(new_experience)

        # Calculate loss
        loss = loss.mean()

        # Add regularizer that penalizes high likelihood for the entire sequence
        loss_p = - (1 / agent_likelihood).mean()
        loss += 5 * 1e3 * loss_p

        # Calculate gradients and make an update to the network weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Convert to numpy arrays so that we can print them
        augmented_likelihood = augmented_likelihood.data.cpu().numpy()
        agent_likelihood = agent_likelihood.data.cpu().numpy()

        # Print some information for this step
        time_elapsed = (time.time() - start_time) / 3600
        time_left = (time_elapsed * ((n_steps - step) / (step + 1)))
        print("\n       Step {}   Fraction valid SMILES: {:4.1f}  Time elapsed: {:.2f}h Time left: {:.2f}h".format(
              step, fraction_valid_smiles(smiles) * 100, time_elapsed, time_left))
        print("  Agent    Prior   Target   Score             SMILES")
        for i in range(10):
            print(" {:6.2f}   {:6.2f}  {:6.2f}  {:6.2f}     {}".format(agent_likelihood[i],
                                                                       prior_likelihood[i],
                                                                       augmented_likelihood[i],
                                                                       score[i],
                                                                       smiles[i]))
    if not save_dir:
        save_dir = '/content/drive/My Drive/De NovoDrug/results/run_' + time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime())
    os.makedirs(save_dir)

    experience.print_memory(os.path.join(save_dir, "memory"))
    #Save Agent's trained weights 
    torch.save(Agent.rnn.state_dict(), os.path.join(save_dir, 'Agent.ckpt'))
    #After training sample from agent model 
    seqs, agent_likelihood, entropy = Agent.sample(256)
    prior_likelihood, _ = Prior.likelihood(Variable(seqs))
    prior_likelihood = prior_likelihood.data.cpu().numpy()
    smiles = seq_to_smiles(seqs, voc)
    score = scoring_function(smiles)
    with open(os.path.join(save_dir, "sampled"), 'w') as f:
        f.write("SMILES Score PriorLogP\n")
        for smiles, score, prior_likelihood in zip(smiles, score, prior_likelihood):
            f.write("{} {:5.2f} {:6.2f}\n".format(smiles, score, prior_likelihood))

if __name__ == "__main__":
    train_agent()

Model initialized, starting training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



       Step 0   Fraction valid SMILES: 90.6  Time elapsed: 0.00h Time left: 12.00h
  Agent    Prior   Target   Score             SMILES
 -17.56   -17.56   41.99    0.99     CC(=O)N1CCN(C(=O)CCOc2ccc(C#N)cc2)CC1
 -18.10   -18.10   41.23    0.99     N#Cc1cccc(CNC(=O)N2CCC3(CCCCC3)C2)c1
 -20.56   -20.56   37.82    0.97     C[C@@H](C(=O)N1CCC(C(=O)c2ccc3ccccc3c2)CC1)c1ccsc1
 -22.27   -22.27   35.57    0.96     CCc1ccc([C@H]2Nc3ccccc3C(=O)N2NC(=O)c2ccc(Cl)cc2)cc1
 -18.43   -18.43   40.95    0.99     C[C@@H](CNC(=O)COc1ccc(Cl)cc1Cl)c1ccncc1
 -17.25   -17.25   41.77    0.98     CNC(=O)C1CC[NH+](CC(=O)Nc2cc(F)ccc2F)CC1
 -25.19   -25.19   32.93    0.97     C1CCCC[C@H]1Nc1ccc(F)cc1
 -19.91   -19.91   38.32    0.97     [NH3+]Cc1ccc(S(=O)(=O)c2cc(Br)ccc2F)cc1
 -22.00   -22.00   37.36    0.99     COc1ccc(CCNS(=O)(=O)N2CCN(C(=O)C(C)C)CC2)cc1OC
 -19.39   -19.39   39.20    0.98     Cc1ccc(N2C[C@@H](C(=O)N3CCCCc4ccccc43)CC2=O)cc1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



Best score in memory: 1.00

       Step 1   Fraction valid SMILES: 82.8  Time elapsed: 0.01h Time left: 11.97h
  Agent    Prior   Target   Score             SMILES
 -19.99   -19.82   38.57    0.97     CN1C(=O)c2ccccc2[C@H](C(=O)N2Cc3ccccc3C[C@H]2C)C1=O
 -31.03   -31.05   26.38    0.96     Br/C(/C=C\c1ccccc1)=C1/C(=N)NC(=O)N(c2cccc(Cl)c2)C1=O
 -22.04   -21.56   37.11    0.98     CCOC[C@H]1CCCN(c2nc(-c3ccccc3Br)no2)C1
 -22.66   -22.61   35.19    0.96     COc1ccc(-c2cccc(C=O)c2)cc1C=O
 -31.10   -30.55  -30.55    0.00     CC(=O)O[C@@H]1C[C@@H]2[C@H](CC(=O)[C@H]3C(=O)N(CCOC)C(=O)[C@@H]3[C@@H](O)[C@]21C
 -22.87   -23.43  -23.43    0.00     Cc1nc(COc2ccc(C(=O)NNC(=O)CCOCC(C)C)cc2)s1
 -20.65   -21.17   38.06    0.99     Cn1cc(C(=O)Nc2ccc(C(F)(F)F)cc2)c(C(=O)N2CCCCC2)n1
 -18.55   -17.95   41.45    0.99     Cc1n[nH]c(C)c1CNC(=O)Cc1cccc(OCC#N)c1
 -31.98   -34.20  -34.20    0.00     Cc1cc(C(C)(C)C)cc(C)cc1S(=O)(=O)N1CCC([C@@]2(O)CCCN(C(=O)c3ccc4c(c3)OCCO4)CC2)CC1
 -18.34   -18.16   41.23    0.99 

...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



Best score in memory: 1.00

       Step 177   Fraction valid SMILES: 93.8  Time elapsed: 1.15h Time left: 18.30h
  Agent    Prior   Target   Score             SMILES
 -24.20   -24.89   32.86    0.96     CCN(Cc1ccccn1)c1ncnc(NNC(=O)CNC(C)=O)c1[N+](=O)[O-]
 -15.10   -24.58   33.97    0.98     FC(F)(F)Oc1ccc(Cl)c(CNc2cccc(I)c2)c1
 -19.18   -27.50  -27.50    0.00     CCOC(=O)CSc1nnc(N(C)Cc2ccccc2)n1
 -18.36   -24.16   34.80    0.98     O=C(OCc1ccc(F)c(F)c1)C(=O)Nc1ccc(OCC(F)(F)F)cc1
 -14.56   -18.37   39.39    0.96     CCC[NH2+][C@H](C(C)C)C1CCC1
 -22.96   -22.15   36.22    0.97     CC(C)S(=O)(=O)NCC(=O)N[C@H]1CCC[C@@H](C)[C@@H]1C
 -14.27   -17.15   41.76    0.98     ClCC1([C@H](O)c2ccccc2)CCC1
 -17.94   -17.79   41.06    0.98     CCOC(=O)c1ccsc1NC(=O)[C@H]1COc2ccccc2C1
 -19.42   -22.12   35.65    0.96     O=C1O[C@H](Cc2ccccc2)C[C@@H]1Cc1ccncc1
 -22.53   -28.27   30.94    0.99     COCCNCC1=C(C(=O)NCc2cccnc2)c2ccccc21


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



Best score in memory: 1.00

       Step 178   Fraction valid SMILES: 93.8  Time elapsed: 1.16h Time left: 18.25h
  Agent    Prior   Target   Score             SMILES
 -21.79   -21.62   37.63    0.99     C[C@H]1CCC[C@H]([NH+](C)CCNC(=O)Nc2ccc(C(N)=O)cc2)C1
 -17.84   -20.06   36.75    0.95     CCOc1ncncc1-c1nc(C)no1
 -30.61   -30.98   26.61    0.96     Cc1c([N+](=O)[O-])ccc(S(=O)(=O)NCCC(=O)NCC(F)(F)F)c1O
 -21.65   -22.62   37.38    1.00     COc1ccc(N[C@H](C)C(=O)NCCCSC)cc1
 -19.43   -21.57   36.26    0.96     Cc1cnccc1NC(=O)C[S@@](=O)Cc1ncn(-c2ccccc2)n1
 -16.01   -17.15   41.51    0.98     CCNC(=O)CNC(=O)N[C@H]1CCC[C@@H](C(F)(F)F)C1
 -28.18   -37.81   21.71    0.99     COc1ccc(OC)c([C@@H]2CN(CC[NH+](C)C)CCN(C(=O)Cc3ccccc3)C[C@@H]2C)c1
 -18.03   -19.73   39.33    0.98     Cc1cc(C(=O)NNC(=O)Cc2ccc(F)cc2)ccc1Cl
 -18.56   -18.91   40.21    0.99     CN(C)C(=O)c1ccc(NC(=O)N2CCCC[C@@H]2c2ccccc2)cc1
 -16.84   -19.64   39.88    0.99     Cc1csc(CNC(=O)c2cc(NC(=O)OC(C)(C)C)ccc2F)n1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
